In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd

In [3]:
# import labels for the shapes
medical_shape_labels = np.genfromtxt('../FiltrationsForGNNs/Medical Dataset/shape_labels.csv', delimiter=',', skip_header=1)
medical_shape_labels = medical_shape_labels.astype(int)[:,2]
print(medical_shape_labels)
print(len(medical_shape_labels))

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
162


In [7]:
num_samples = 162 # currently set to full dataset

# Generate random indices
random_indices = np.random.choice(len(medical_shape_labels), size=num_samples, replace=False)
train_indices = random_indices[:100]
validation_indices=random_indices[100:]
base = '../FiltrationsForGNNs/Medical Dataset/'
#Select the corresponding data and labels
medical_laplacians_train = []
medical_vr_persistence_images_train = []
medical_abstract_persistence_images_train = []
medical_selected_labels_train = []

for i in train_indices:
    medical_laplacians_train.append(np.genfromtxt(f'{base}/shape_{i}_laplacian.csv', delimiter=',', skip_header=0))
    medical_vr_persistence_images_train.append(np.genfromtxt(f'{base}/shape_{i}_vr_persistence_image.csv', delimiter=',', skip_header=0))
    medical_abstract_persistence_images_train.append(np.genfromtxt(f'{base}/shape_{i}_abstract_persistence_image.csv', delimiter=',', skip_header=0))
    medical_selected_labels_train.append(medical_shape_labels[i])

# Convert selected labels to NumPy array
medical_selected_labels_train = np.array(medical_selected_labels_train)

# Print a summary
print(f"Randomly selected {num_samples} samples.")
print(f"Shape of laplacians: {np.array(medical_laplacians_train).shape}")
print(f"Shape of VR persistence images: {np.array(medical_vr_persistence_images_train).shape}")
print(f"Shape of abstract persistence images: {np.array(medical_abstract_persistence_images_train).shape}")
print(f"Shape of selected labels: {medical_selected_labels_train.shape}")

Randomly selected 162 samples.
Shape of laplacians: (100, 1000, 1000)
Shape of VR persistence images: (100, 100, 100)
Shape of abstract persistence images: (100, 100, 100)
Shape of selected labels: (100,)


In [8]:
medical_laplacians_validation = []
medical_vr_persistence_images_validation = []
medical_abstract_persistence_images_validation = []
medical_selected_labels_validation = []

for i in validation_indices:
    medical_laplacians_validation.append(np.genfromtxt(f'{base}/shape_{i}_laplacian.csv', delimiter=',', skip_header=0))
    medical_vr_persistence_images_validation.append(np.genfromtxt(f'{base}/shape_{i}_vr_persistence_image.csv', delimiter=',', skip_header=0))
    medical_abstract_persistence_images_validation.append(np.genfromtxt(f'{base}/shape_{i}_abstract_persistence_image.csv', delimiter=',', skip_header=0))
    medical_selected_labels_validation.append(medical_shape_labels[i])

# Convert selected labels to NumPy array
medical_selected_labels_validation = np.array(medical_selected_labels_validation)

print(f"Randomly selected {num_samples} samples.")
print(f"Shape of laplacians: {np.array(medical_laplacians_validation).shape}")
print(f"Shape of VR persistence images: {np.array(medical_vr_persistence_images_validation).shape}")
print(f"Shape of abstract persistence images: {np.array(medical_abstract_persistence_images_validation).shape}")
print(f"Shape of selected labels: {medical_selected_labels_validation.shape}")

Randomly selected 162 samples.
Shape of laplacians: (62, 1000, 1000)
Shape of VR persistence images: (62, 100, 100)
Shape of abstract persistence images: (62, 100, 100)
Shape of selected labels: (62,)


In [9]:
class ShapeDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = [torch.tensor(d, dtype=torch.float32).unsqueeze(0) for d in data]
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


In [10]:
class CNN(nn.Module):
    def __init__(self, input_shape, num_classes=2):
        super(CNN, self).__init__()
        # Convolutional Layers
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        
        # Pooling Layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Adaptive Pooling to resize to 100x100
        self.adaptive_pool = nn.AdaptiveAvgPool2d((100, 100))
        
        # Dynamically calculate input size to fc1
        self.feature_size = self._get_feature_size(input_shape)
        
        # Fully Connected Layers
        self.fc1 = nn.Linear(self.feature_size, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, num_classes)

    def _get_feature_size(self, input_shape):
        # Create a dummy input to calculate size after conv and pooling
        dummy_input = torch.zeros(1, 1, *input_shape)
        x = self.pool(F.relu(self.conv1(dummy_input)))
        x = self.pool(F.relu(self.conv2(x)))
        
        # Apply adaptive pooling to get 100x100 size
        x = self.adaptive_pool(x)
        return x.numel()  # Number of elements after flattening

    def forward(self, x):
        # Apply convolutional layers with pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        
        # Apply adaptive pooling to resize to 100x100
        x = self.adaptive_pool(x)
        
        # Flatten and pass through fully connected layers
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


In [11]:
class DualInputCNN(nn.Module):
    def __init__(self, input_shape1, input_shape2, num_classes=2):
        super(DualInputCNN, self).__init__()

        # Laplacian input path with additional pooling to reduce to 100x100
        self.conv1_lap = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2_lap = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool_lap = nn.MaxPool2d(2, 2)  # Reduce spatial dimensions
        self.adaptive_pool_lap = nn.AdaptiveAvgPool2d((100, 100))  # Resize to 100x100
        
        # Persistence image input path (no pooling)
        self.conv1_pers = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2_pers = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.adaptive_pool_pers = nn.AdaptiveAvgPool2d((100, 100))  # Resize to 100x100
        
        # Fully connected layers
        self.fc1 = nn.Linear(32 * 100 * 100 + 32 * 100 * 100, 128)  # Adjusted for 100x100 input
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x1, x2):
        # Laplacians path (downsampling to 100x100)
        x1 = F.relu(self.conv1_lap(x1))
        x1 = self.pool_lap(x1)  # First pool: 250x250 -> 125x125
        x1 = F.relu(self.conv2_lap(x1))
        x1 = self.pool_lap(x1)  # Second pool: 125x125 -> 62x62
        x1 = self.adaptive_pool_lap(x1)  # Resize to 100x100
        
        # Persistence images path (no pooling)
        x2 = F.relu(self.conv1_pers(x2))
        x2 = F.relu(self.conv2_pers(x2))
        x2 = self.adaptive_pool_pers(x2)  # Ensure persistence images are 100x100
        
        # Concatenate along dim=1 (channels)
        x = torch.cat((x1, x2), dim=1)  # Concatenates the outputs along the channel axis

        # Flatten for fully connected layer
        x = torch.flatten(x, start_dim=1)
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on {device}")

Training on cpu


In [13]:
def train_single_input(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    tk0 = tqdm(train_loader, total=len(train_loader))
    for batch_idx, (data, target) in enumerate(tk0):
        data, target = data.to(device), target.to(device)
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Forward pass with single input
        output = model(data)  # Forward through the single-input model
        
        # Compute loss
        loss = criterion(output, target)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Update loss and accuracy
        train_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

        # Update progress bar
        tk0.set_postfix(loss=loss.item())

    avg_loss = train_loss / len(train_loader)
    accuracy = 100. * correct / total
    return avg_loss, accuracy


def test_single_input(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)  # Forward through the single-input model
            loss = criterion(output, target)
            
            test_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()
            
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
    
    avg_loss = test_loss / len(test_loader)
    accuracy = 100. * correct / total
    
    # Calculate additional metrics
    precision = precision_score(all_targets, all_preds, average='weighted')
    recall = recall_score(all_targets, all_preds, average='weighted')
    f1 = f1_score(all_targets, all_preds, average='weighted')
    
    return avg_loss, accuracy, precision, recall, f1


In [14]:
def train_and_test_single_input(data_type, data, labels, input_shape):
    # Create dataset and split into train/test sets
    dataset = ShapeDataset(data, labels)
    train_data, test_data, train_labels, test_labels = train_test_split(
        dataset.data, dataset.labels, test_size=0.2, random_state=42
    )

    # Convert to custom Dataset format for train and test sets
    train_dataset = torch.utils.data.TensorDataset(torch.stack(train_data), train_labels)
    test_dataset = torch.utils.data.TensorDataset(torch.stack(test_data), test_labels)

    # Create DataLoaders
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Define CNN model with input_shape
    model = CNN(input_shape=input_shape, num_classes=len(set(labels))).to(device)

    # Define optimizer and loss function
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()  # For multi-class classification

    epoch_results = []

    # Training loop
    num_epochs = 10
    for epoch in range(1, num_epochs + 1):
        print(f"Training model for {data_type} - Epoch {epoch}/{num_epochs}")
        train_loss, accuracy = train_single_input(model, device, train_dataloader, optimizer, criterion, epoch)
        test_loss, accuracy, precision, recall, f1 = test_single_input(model, device, test_dataloader, criterion)

        # Store results
        epoch_results.append({
            'Epoch': epoch,
            'Test Loss': test_loss,
            'Test Accuracy (%)': accuracy,
            'Test Precision (%)': precision * 100,
            'Test Recall (%)': recall * 100,
            'Test F1 Score': f1
        })

    # Convert results to DataFrame for tabular output
    epoch_results_df = pd.DataFrame(epoch_results)
    print(f"\nTesting results for {data_type}:")
    print(epoch_results_df)
    return model


In [15]:
def train_and_test_dual_input(data_type, data1, data2, labels, input_shape1, input_shape2):
    dataset1 = [torch.tensor(d, dtype=torch.float32).unsqueeze(0) for d in data1]  # Shape [1, 100, 100]
    dataset2 = [torch.tensor(d, dtype=torch.float32).unsqueeze(0) for d in data2]  # Shape [1, 100, 100]
    labels = torch.tensor(labels, dtype=torch.long)

    train_data1, test_data1, train_data2, test_data2, train_labels, test_labels = train_test_split(
        dataset1, dataset2, labels, test_size=0.2, random_state=42
    )

    train_dataset = torch.utils.data.TensorDataset(
        torch.stack(train_data1), torch.stack(train_data2), train_labels
    )
    test_dataset = torch.utils.data.TensorDataset(
        torch.stack(test_data1), torch.stack(test_data2), test_labels
    )

    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

    model = DualInputCNN(input_shape1=input_shape1, input_shape2=input_shape2, num_classes=len(set(labels))).to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    epoch_results = []

    num_epochs = 10
    for epoch in range(1, num_epochs + 1):
        print(f"Training model for {data_type} - Epoch {epoch}/5")
        
        # Training step
        train_loss, train_accuracy = train_dual_input(model, device, train_dataloader, optimizer, criterion, epoch)
        
        # Testing step
        test_loss, test_accuracy, precision, recall, f1 = test_dual_input(model, device, test_dataloader, criterion)
        
        # Storing results
        epoch_results.append({
            'Epoch': epoch,
            'Test Loss': test_loss,
            'Test Accuracy (%)': test_accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1 Score': f1
        })

    # Convert results to DataFrame for tabular output
    epoch_results_df = pd.DataFrame(epoch_results)
    print(f"\nTesting results for {data_type}:")
    print(epoch_results_df)
    return model


In [16]:
def train_dual_input(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    tk0 = tqdm(train_loader, total=len(train_loader))
    for batch_idx, (data1, data2, target) in enumerate(tk0):
        data1, data2, target = data1.to(device), data2.to(device), target.to(device)
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Forward pass with dual input
        output = model(data1, data2)  # Forward through the dual-input model
        
        # Compute loss
        loss = criterion(output, target)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Update loss and accuracy
        train_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

        # Update progress bar
        tk0.set_postfix(loss=loss.item())

    avg_loss = train_loss / len(train_loader)
    accuracy = 100. * correct / total
    return avg_loss, accuracy


def test_dual_input(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    all_labels = []
    all_preds = []
    
    with torch.no_grad():
        for data1, data2, target in test_loader:
            data1, data2, target = data1.to(device), data2.to(device), target.to(device)
            output = model(data1, data2)  # Forward through the dual-input model
            loss = criterion(output, target)
            
            test_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()

            # Collect all labels and predictions for additional metrics
            all_labels.extend(target.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())
    
    avg_loss = test_loss / len(test_loader)
    accuracy = 100. * correct / total
    
    # Calculate additional metrics
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')

    # Return test loss, accuracy, and additional metrics
    return avg_loss, accuracy, precision, recall, f1


In [17]:
# Train and test for Laplacians (1000x1000 input)
lap = train_and_test_single_input("Laplacians", medical_laplacians_train, medical_selected_labels_train, input_shape=(1000, 1000))

Training model for Laplacians - Epoch 1/10


  0%|          | 0/5 [00:00<?, ?it/s]

Training model for Laplacians - Epoch 2/10


  0%|          | 0/5 [00:00<?, ?it/s]

Training model for Laplacians - Epoch 3/10


...

Training model for Laplacians - Epoch 9/10


  0%|          | 0/5 [00:00<?, ?it/s]

Training model for Laplacians - Epoch 10/10


  0%|          | 0/5 [00:00<?, ?it/s]


Testing results for Laplacians:
   Epoch  Test Loss  Test Accuracy (%)  Test Precision (%)  Test Recall (%)  \
0      1   0.665645               45.0           78.611111             45.0   
1      2   0.641405               65.0           42.250000             65.0   
2      3   0.484549               65.0           42.250000             65.0   
3      4   0.432868               65.0           42.250000             65.0   
4      5   0.387218               65.0           42.250000             65.0   
5      6   0.345222               65.0           42.250000             65.0   
6      7   0.296582               80.0           84.705882             80.0   
7      8   0.262442               95.0           95.357143             95.0   
8      9   0.252081              100.0          100.000000            100.0   
9     10   0.204030              100.0          100.000000            100.0   

   Test F1 Score  
0       0.369333  
1       0.512121  
2       0.512121  
3       0.512121  
4 

In [19]:
dual_vr = train_and_test_dual_input("Laplacians + VR Persistence Images", medical_laplacians_train, medical_vr_persistence_images_train, medical_selected_labels_train, (1000, 1000), (100, 100))

Training model for Laplacians + VR Persistence Images - Epoch 1/5


  0%|          | 0/5 [00:00<?, ?it/s]

Training model for Laplacians + VR Persistence Images - Epoch 2/5


c:\Users\alanp\anaconda3\envs\FiltrationsForGNNs\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


  0%|          | 0/5 [00:00<?, ?it/s]

...

Training model for Laplacians + VR Persistence Images - Epoch 9/5


  0%|          | 0/5 [00:00<?, ?it/s]

Training model for Laplacians + VR Persistence Images - Epoch 10/5


  0%|          | 0/5 [00:00<?, ?it/s]


Testing results for Laplacians + VR Persistence Images:
   Epoch  Test Loss  Test Accuracy (%)  Precision  Recall  F1 Score
0      1  73.238319               65.0   0.422500    0.65  0.512121
1      2   7.220216               60.0   0.813333    0.60  0.583838
2      3   3.280545               75.0   0.819444    0.75  0.700717
3      4   0.115575               95.0   0.953571    0.95  0.949003
4      5   0.251940               95.0   0.956250    0.95  0.950667
5      6   0.105606              100.0   1.000000    1.00  1.000000
6      7   0.064116              100.0   1.000000    1.00  1.000000
7      8   0.052204              100.0   1.000000    1.00  1.000000
8      9   0.187437               90.0   0.913333    0.90  0.895238
9     10   0.125898               95.0   0.956250    0.95  0.950667


In [20]:
dual_AP = train_and_test_dual_input("Laplacians + Abstract Persistence Images", medical_laplacians_train, medical_abstract_persistence_images_train, medical_selected_labels_train, (1000, 1000), (100, 100))

Training model for Laplacians + Abstract Persistence Images - Epoch 1/5


  0%|          | 0/5 [00:00<?, ?it/s]

Training model for Laplacians + Abstract Persistence Images - Epoch 2/5


  0%|          | 0/5 [00:00<?, ?it/s]

Training model for Laplacians + Abstract Persistence Images - Epoch 3/5


...

Training model for Laplacians + Abstract Persistence Images - Epoch 9/5


  0%|          | 0/5 [00:00<?, ?it/s]

Training model for Laplacians + Abstract Persistence Images - Epoch 10/5


  0%|          | 0/5 [00:00<?, ?it/s]


Testing results for Laplacians + Abstract Persistence Images:
   Epoch  Test Loss  Test Accuracy (%)  Precision  Recall  F1 Score
0      1   7.720898               60.0   0.410526    0.60  0.487500
1      2   3.239947               65.0   0.608333    0.65  0.581004
2      3   0.751923               75.0   0.744048    0.75  0.745014
3      4   0.610184               75.0   0.744048    0.75  0.745014
4      5   0.585752               80.0   0.800000    0.80  0.790476
5      6   0.673741               65.0   0.608333    0.65  0.581004
6      7   0.842980               65.0   0.608333    0.65  0.581004
7      8   0.525397               85.0   0.895000    0.85  0.853453
8      9   0.669665               65.0   0.608333    0.65  0.581004
9     10   0.435691               80.0   0.800000    0.80  0.790476


In [21]:
def validation_single(model, data):
    test_dataloader = torch.utils.data.DataLoader(data, batch_size=32, shuffle=False)

    epoch_results = []
    # Testing step
    test_loss, test_accuracy, precision, recall, f1 = test_single_input(model, device, test_dataloader, nn.CrossEntropyLoss())


    # Storing results
    epoch_results.append({
    'Test Loss': test_loss,
    'Test Accuracy (%)': test_accuracy,
    'Precision': precision,
    'Recall': recall,
    'F1 Score': f1
    })

    # Convert results to DataFrame for tabular output
    epoch_results_df = pd.DataFrame(epoch_results)
    y = []
    with torch.no_grad():
        for data, target in test_dataloader:
            data, target = data.to(device), target.to(device)
            output = lap(data)  # Forward through the single-input model
            _, predicted = output.max(1)
            y.append(predicted)
    return epoch_results_df, y

In [22]:
data_lap = ShapeDataset(medical_laplacians_validation, medical_selected_labels_validation)

In [23]:
results,output = validation_single(lap, data_lap)

In [25]:
results, output

(   Test Loss  Test Accuracy (%)  Precision    Recall  F1 Score
 0   0.215015          93.548387    0.94086  0.935484  0.932854,
 [tensor([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
          1, 1, 1, 0, 1, 0, 1, 1]),
  tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
          0, 0, 0, 1, 0, 0])])

In [ ]:
def validation_dual(model, data1, data2, label):
    dataset1 = [torch.tensor(d, dtype=torch.float32).unsqueeze(0) for d in data1]  # Shape [1, 100, 100]
    dataset2 = [torch.tensor(d, dtype=torch.float32).unsqueeze(0) for d in data2]  # Shape [1, 100, 100]
    labels = torch.tensor(label, dtype=torch.long)
    test_dataset = torch.utils.data.TensorDataset(
        torch.stack(dataset1), torch.stack(dataset2), labels
    )
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)
    test_loss, test_accuracy, precision, recall, f1 = test_dual_input(model, device, test_dataloader, nn.CrossEntropyLoss())
    epoch_results = []

    # Storing results
    epoch_results.append({
        'Test Loss': test_loss,
        'Test Accuracy (%)': test_accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    })

    # Convert results to DataFrame for tabular output
    epoch_results_df = pd.DataFrame(epoch_results)
    y = []
    with torch.no_grad():
        for data1, data2, target in test_dataloader:
            data1, data2, target = data1.to(device), data2.to(device), target.to(device)
            output = dual_vr(data1, data2)  # Forward through the dual-input model
            _, predicted = output.max(1)
            y.append(predicted)
    return epoch_results_df, y

In [27]:
results_vr, output_vr = validation_dual(dual_vr, medical_laplacians_validation, medical_vr_persistence_images_validation, medical_selected_labels_validation)

In [28]:
results_vr, output_vr

(   Test Loss  Test Accuracy (%)  Precision    Recall  F1 Score
 0   0.320115          88.709677    0.91871  0.887097  0.891408,
 [tensor([0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,
          1, 1, 1, 0, 1, 0, 1, 1]),
  tensor([1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0,
          0, 0, 0, 1, 1, 1])])

In [29]:
results_ap, output_ap = validation_dual(dual_AP, medical_laplacians_validation, medical_abstract_persistence_images_validation, medical_selected_labels_validation)

In [30]:
results_ap, output_ap

(   Test Loss  Test Accuracy (%)  Precision    Recall  F1 Score
 0   0.974446          69.354839   0.500264  0.693548   0.58126,
 [tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0]),
  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0])])